# 0717 과제

### - 공공데이터 DB 저장

---

In [10]:
import requests

resp = requests.get("http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?serviceKey=Nl0FFvhW%2BGP2f7OFZT0z%2FOPn1c0Q%2FPeuln%2Fn8Z4FBbix6OxUWDa9nfTcQaZDnsTAlSBT08UektJEsEJTaN3Q0w%3D%3D&numOfRows=10&pageSize=10&pageNo=1&startPage=1&stationName=%EC%A2%85%EB%A1%9C%EA%B5%AC&dataTerm=DAILY&ver=1.3")

In [3]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(resp.text, "lxml")

In [5]:
parsed = dom.find_all("item")

In [17]:
# item tag 검색 결과

parsed

[<item>
 <datatime>2018-07-18 01:00</datatime>
 <mangname>도시대기</mangname>
 <so2value>0.003</so2value>
 <covalue>0.2</covalue>
 <o3value>0.002</o3value>
 <no2value>0.019</no2value>
 <pm10value>33</pm10value>
 <pm10value24>30</pm10value24>
 <pm25value>26</pm25value>
 <pm25value24>27</pm25value24>
 <khaivalue>79</khaivalue>
 <khaigrade>2</khaigrade>
 <so2grade>1</so2grade>
 <cograde>1</cograde>
 <o3grade>1</o3grade>
 <no2grade>1</no2grade>
 <pm10grade>1</pm10grade>
 <pm25grade>2</pm25grade>
 <pm10grade1h>2</pm10grade1h>
 <pm25grade1h>2</pm25grade1h>
 </item>, <item>
 <datatime>2018-07-17 24:00</datatime>
 <mangname>도시대기</mangname>
 <so2value>0.004</so2value>
 <covalue>0.2</covalue>
 <o3value>0.002</o3value>
 <no2value>0.023</no2value>
 <pm10value>27</pm10value>
 <pm10value24>30</pm10value24>
 <pm25value>25</pm25value>
 <pm25value24>27</pm25value24>
 <khaivalue>79</khaivalue>
 <khaigrade>2</khaigrade>
 <so2grade>1</so2grade>
 <cograde>1</cograde>
 <o3grade>1</o3grade>
 <no2grade>1</no2grad

---

In [151]:
# 수치 tuple 형태로 저장

contentDic = {}

for i, j in enumerate(parsed):
    temp = []
    temp = j.get_text().split("\n")
    temp = temp[1:len(temp)-1]
    contentList = [i]
    contentList += temp
    contentDic[i] = tuple(contentList)

In [152]:
# 수치 contentList에 저장

contentList = []

for content in contentDic.values():
    contentList.append(content)

In [153]:
# tag 이름 저장
typeList = ["INT PRIMARY KEY", "TEXT", "TEXT", "REAL", "REAL", "REAL", "REAL", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT", "INT"]

tagList = ['id']

for j in parsed[0]:
    if j.name:
        tagList.append(j.name)

In [154]:
# tagList 결과

tagList

['id',
 'datatime',
 'mangname',
 'so2value',
 'covalue',
 'o3value',
 'no2value',
 'pm10value',
 'pm10value24',
 'pm25value',
 'pm25value24',
 'khaivalue',
 'khaigrade',
 'so2grade',
 'cograde',
 'o3grade',
 'no2grade',
 'pm10grade',
 'pm25grade',
 'pm10grade1h',
 'pm25grade1h']

In [147]:
import sqlite3

In [148]:
urban_air = "CREATE TABLE UrbanAir("

for i in range(0, len(tagList)-1):
    urban_air += ' ' + tagList[i] +' ' + typeList[i] + ','

urban_air += ' '+ tagList[i+1] +' ' + typeList[i+1] +')'

print(urban_air)

CREATE TABLE UrbanAir( id INT PRIMARY KEY, datatime TEXT, mangname TEXT, so2value TEXT, covalue TEXT, o3value TEXT, no2value TEXT, pm10value TEXT, pm10value24 TEXT, pm25value TEXT, pm25value24 TEXT, khaivalue TEXT, khaigrade TEXT, so2grade TEXT, cograde TEXT, o3grade TEXT, no2grade TEXT, pm10grade TEXT, pm25grade TEXT, pm10grade1h TEXT, pm25grade1h TEXT)


In [156]:
con = sqlite3.connect("air_pollution.db")

with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS UrbanAir")
    cur.execute(urban_air)
    cur.executemany("INSERT INTO UrbanAir VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", contentList)

---